In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE202806"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE202806"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE202806.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE202806.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE202806.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic analyses of NF1-mutant melanoma"
!Series_summary	"RNA analysis of 770 genes (Pan Cancer IO 360) related to the tumor microenvironment on NF1-MUT and matched NF1-WT samples for reference."
!Series_overall_design	"Multiplex gene expression analysis covers 770 genes from 24 different immune cell types and 48 gene-derived signatures measuring biological variables crucial to the tumor-immune interaction including cell proliferation, angiogenesis and immune inhibitory mechanisms, among others."
Sample Characteristics Dictionary:
{0: ['tissue: Melanoma'], 1: ['nf1 status: WT', 'nf1 status: MUT']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, we see this dataset contains gene expression data for 770 genes
# related to tumor microenvironment, so it's suitable for our analysis.
is_gene_available = True

# 2.1 Data Availability
# Examining the sample characteristics dictionary:
# - Key 0 shows 'tissue: Melanoma' which is constant for all samples (not useful for trait)
# - Key 1 shows 'nf1 status: WT' and 'nf1 status: MUT' which can be used as our trait variable

# For Melanoma trait, we'll use NF1 mutation status as the binary trait
trait_row = 1  # nf1 status

# Age and gender information are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert NF1 mutation status to binary values."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert NF1 status to binary
    if value.upper() == 'MUT':
        return 1  # Mutated
    elif value.upper() == 'WT':
        return 0  # Wild type
    else:
        return None

def convert_age(value):
    """Convert age values to numeric format."""
    # Not applicable for this dataset
    return None

def convert_gender(value):
    """Convert gender values to binary format."""
    # Not applicable for this dataset
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort,
                             info_path=json_path,
                             is_gene_available=is_gene_available,
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features_df)
    print("Clinical Features Preview:", preview)
    
    # Save the clinical data
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview: {'GSM6133552': [0.0], 'GSM6133553': [0.0], 'GSM6133554': [0.0], 'GSM6133555': [0.0], 'GSM6133556': [0.0], 'GSM6133557': [1.0], 'GSM6133558': [1.0], 'GSM6133559': [1.0], 'GSM6133560': [1.0], 'GSM6133561': [1.0], 'GSM6133562': [0.0], 'GSM6133563': [1.0], 'GSM6133564': [1.0], 'GSM6133565': [1.0], 'GSM6133566': [1.0], 'GSM6133567': [1.0], 'GSM6133568': [1.0], 'GSM6133569': [1.0], 'GSM6133570': [1.0], 'GSM6133571': [1.0], 'GSM6133572': [0.0], 'GSM6133573': [0.0], 'GSM6133574': [0.0], 'GSM6133575': [0.0], 'GSM6133576': [0.0], 'GSM6133577': [0.0], 'GSM6133578': [0.0], 'GSM6133579': [0.0], 'GSM6133580': [0.0], 'GSM6133581': [1.0], 'GSM6133582': [0.0], 'GSM6133583': [0.0], 'GSM6133584': [1.0], 'GSM6133585': [1.0], 'GSM6133586': [0.0], 'GSM6133587': [0.0], 'GSM6133588': [1.0], 'GSM6133589': [0.0], 'GSM6133590': [0.0], 'GSM6133591': [0.0], 'GSM6133592': [1.0], 'GSM6133593': [0.0], 'GSM6133594': [0.0], 'GSM6133595': [1.0], 'GSM6133596': [0.0], 'GSM6133597': [1.0], 'GSM61

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.

Gene data extraction result:
Number of rows: 784
First 20 gene/probe identifiers:
Index(['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1',
       'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B',
       'API5', 'APLNR', 'APOE', 'APOL6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the data are human gene symbols (A2M, ABCF1, ACVR1C, etc.)
# These are standard HGNC gene symbols that don't require mapping to other identifiers

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Extract clinical features using the clinical data from step 1
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features using the convert_trait function from step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # From step 2
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 5. Determine if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from juvenile myositis (JM) and childhood-onset lupus (cSLE) skin biopsies."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Top 10 gene indices before normalization: ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC']
Top 10 gene indices after normalization: ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC']
Shape of normalized gene data: (762, 52)
Saved normalized gene data to ../../output/preprocess/Melanoma/gene_data/GSE202806.csv
Saved clinical data to ../../output/preprocess/Melanoma/clinical_data/GSE202806.csv
Shape of linked data: (52, 763)
Shape of linked data after handling missing values: (52, 763)


For the feature 'Melanoma', the least common label is '1.0' with 24 occurrences. This represents 46.15% of the dataset.
The distribution of the feature 'Melanoma' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Melanoma/GSE202806.csv
